Install amplpy and other packages

In [1]:
!pip install -q amplpy ampltools pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.7 MB/s eta 0:00:00


In [2]:
# Google Colab & AMPL integration

MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "6b31af08-ff1f-429f-ad0b-4bb913b68f75"

from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6300.6669 expiring 20231231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read the data using pandas.

In [4]:
import pandas as pd

# Load the data from the Excel file
file_path = "/content/drive/MyDrive/645/Datasets/Food supply chain optimization.xlsx"

# Load data from the sheet containing transportation costs from orchard to preparation centers
prep_center_data = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
preparation_centers = prep_center_data.index.tolist()
cost_orchard_to_prep = prep_center_data['Transportation Cost ($/pound) (Orchard to Preparation Center)'].to_dict()
cost_prep = prep_center_data['Preparation Cost ($/pound)'].to_dict()
capacity_prep = prep_center_data['Monthly Capacity (pounds)'].to_dict()

# Load data from the sheet containing costs from preparation centers to specialty stores and their demand
store_data = pd.read_excel(file_path, sheet_name="sheet2", header=None)
stores = store_data.iloc[0, 1:].values.tolist()
demands = store_data.iloc[-1, 1:].values.tolist()

cost_prep_to_store = {}
for i in range(1, len(store_data)):
    center = store_data.iloc[i, 0]
    if center in preparation_centers:
        costs = dict(zip(stores, store_data.iloc[i, 1:]))
        cost_prep_to_store[center] = costs

# Ensure the indices and lengths align correctly
assert len(stores) == len(demands), "Mismatch in the number of stores and demands"

# Create the demand_store dictionary
demand_store = dict(zip(stores, demands))

# Print to verify the demand_store dictionary
print("Demand for each store:", demand_store)

Demand for each store: {'Organic Orchard': 300, 'Fresh & Local': 500, 'Healthy Pantry': 400, "Season's Harvest": 200}


In [5]:
# Setting up the AMPL model
import amplpy
ampl = amplpy.AMPL()

In [6]:
ampl.eval('''

reset;

# Sets for preparation centers and stores
set PreparationCenters;
set Stores;

# Parameters for costs, capacities, and demands
param Cost_Orchard_to_Prep{PreparationCenters};
param Cost_Prep{PreparationCenters};
param Capacity_Prep{PreparationCenters};
param Cost_Prep_to_Store{PreparationCenters, Stores};
param Demand_Store{Stores};

# Decision variables
var x{PreparationCenters} >= 0;  # Apples to preparation centers
var y{PreparationCenters, Stores} >= 0;  # Apples to stores

# Objective function - Minimize total cost
minimize TotalCost:
    sum{i in PreparationCenters} (Cost_Orchard_to_Prep[i] + Cost_Prep[i]) * x[i] + sum{i in PreparationCenters, j in Stores} Cost_Prep_to_Store[i,j] * y[i,j];

# Constraints
subject to CapacityConstraint{i in PreparationCenters}:
    sum{j in Stores} y[i,j] <= x[i];

subject to DemandConstraint{j in Stores}:
    sum{i in PreparationCenters} y[i,j] >= Demand_Store[j];

subject to PrepCenterCapacity{i in PreparationCenters}:
    x[i] <= Capacity_Prep[i];
''')

In [7]:
# Set the sets for preparation centers and stores in AMPL
ampl.set['PreparationCenters'] = preparation_centers
ampl.set['Stores'] = stores

# Set the parameters for costs, capacities, and demands in AMPL
ampl.param['Cost_Orchard_to_Prep'] = cost_orchard_to_prep
ampl.param['Cost_Prep'] = cost_prep
ampl.param['Capacity_Prep'] = capacity_prep

# Setting the transportation costs from preparation centers to stores in AMPL
for center in preparation_centers:
    for store in stores:
        ampl.param['Cost_Prep_to_Store'][center, store] = cost_prep_to_store[center][store]

ampl.param['Demand_Store'] = demand_store


In [8]:
ampl.eval('''expand;''')

minimize TotalCost:
	0.6*x[1] + 1.2*x[2] + 1.8*x[3] + 0.8*y[1,'Organic Orchard'] + 
	1.1*y[1,'Fresh & Local'] + 0.7*y[1,'Healthy Pantry'] + 
	1.4*y[1,"Season's Harvest"] + 1.2*y[2,'Organic Orchard'] + 
	1.1*y[2,'Fresh & Local'] + 0.5*y[2,'Healthy Pantry'] + 
	1.4*y[2,"Season's Harvest"] + 0.2*y[3,'Organic Orchard'] + 
	1.4*y[3,'Fresh & Local'] + 1.3*y[3,'Healthy Pantry'] + 
	1.7*y[3,"Season's Harvest"];

subject to CapacityConstraint[1]:
	-x[1] + y[1,'Organic Orchard'] + y[1,'Fresh & Local'] + 
	y[1,'Healthy Pantry'] + y[1,"Season's Harvest"] <= 0;

subject to CapacityConstraint[2]:
	-x[2] + y[2,'Organic Orchard'] + y[2,'Fresh & Local'] + 
	y[2,'Healthy Pantry'] + y[2,"Season's Harvest"] <= 0;

subject to CapacityConstraint[3]:
	-x[3] + y[3,'Organic Orchard'] + y[3,'Fresh & Local'] + 
	y[3,'Healthy Pantry'] + y[3,"Season's Harvest"] <= 0;

subject to DemandConstraint['Organic Orchard']:
	y[1,'Organic Orchard'] + y[2,'Organic Orchard'] + 
	y[3,'Organic Orchard'] >= 300;

subject to Dema

In [9]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 3040
0 simplex iterations
 


In [10]:
# Print results
print("Total Cost:", ampl.getObjective('TotalCost').value())
print("Apples to Preparation Centers (x):")
ampl.display('x')
print("Apples from Preparation Centers to Stores (y):")
ampl.display('y')

Total Cost: 3040.0
Apples to Preparation Centers (x):
x [*] :=
1  300
2  500
3  600
;

Apples from Preparation Centers to Stores (y):
y :=
1 'Fresh & Local'      100
1 'Healthy Pantry'       0
1 'Organic Orchard'      0
1 "Season's Harvest"   200
2 'Fresh & Local'      100
2 'Healthy Pantry'     400
2 'Organic Orchard'      0
2 "Season's Harvest"     0
3 'Fresh & Local'      300
3 'Healthy Pantry'       0
3 'Organic Orchard'    300
3 "Season's Harvest"     0
;

